In [62]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__() 
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

    def forward(self, x):
        output, (hidden, cell) = self.rnn(x)
        return output, (hidden, cell)

class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)  # TO DO: Añadir dropout 
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden, cell):
        output, (hidden, cell) = self.rnn(x, (hidden, cell))
        output = self.fc_out(output)
        return output, (hidden, cell)

class Bilinear_Attention(nn.Module):
    def __init__(self, encoder_dim, decoder_dim):
        super(Bilinear_Attention, self).__init__()
        self.W = nn.Linear(decoder_dim, encoder_dim, bias = False)
        
    def forward(self, encoder_out, decoder_hidden):
        decoder_attn = self.W(decoder_hidden)
        print(decoder_attn)
        score = torch.matmul(decoder_attn, encoder_out.transpose(1,2))
        print(score)
        att_weights = torch.softmax(score, dim = 0)
        print(att_weights)
        att = att_weights.matmul(encoder_out)
        return att

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, attention):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder                           
        self.attention = attention
        self.fc = nn.Linear(hiden_dim + attn_dim, hidden_dim)
        self.es_embeddings = torchtext.vocab.FastText(language='es')
        self.M = self.es_embeddings.vectors
        self.M = torch.cat((self.M, torch.zeros((4, self.M.shape[1]))), 0)
        

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        target_len = target.shape[1]
        batch_size = target.shape[0]

        # Tensor para almacenar las salidas del decoder
        outputs = torch.zeros(batch_size, target_len, 985671)
        
        # Primero, la fuente es procesada por el encoder
        encoder_outputs, (hidden, cell) = self.encoder(source)

        # La primera entrada al decoder es el vector <sos>
        x = target[:, 0, :]

        for t in range(1, target_len):
            attention = self.attention(encoder_outputs, hidden)
            new_hidden = torch.cat(hidden,attention, dim = 0)
            new_hidden = self.fc(new_hidden)
            hidden = torch.tanh(new_hidden)
            output, (hidden, cell) = self.decoder(x.unsqueeze(1), hidden, cell)
            outputs[:, t, :] = output.squeeze(1)
            
            teacher_force = random.random() < teacher_forcing_ratio
            if teacher_force:
                x = target[:, t, :]
            else:
                x = torch.matmul(output.squeeze(1), self.M)

        return outputs

In [63]:
# SANITY CHECk
encoder_out = torch.ones((1,3,5))
decoder_hid = torch.ones(5)
print(encoder_out)
print(decoder_hid)

attention_test  = Bilinear_Attention(5,5)


print(attention_test.forward(encoder_out, decoder_hid))

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])
tensor([1., 1., 1., 1., 1.])
tensor([-0.1313, -0.7526,  0.3534, -1.1248, -0.0745],
       grad_fn=<SqueezeBackward4>)
tensor([[-1.7297, -1.7297, -1.7297]], grad_fn=<UnsafeViewBackward0>)
tensor([[1., 1., 1.]], grad_fn=<SoftmaxBackward0>)
tensor([[[3., 3., 3., 3., 3.]]], grad_fn=<TransposeBackward0>)


In [1]:
#Dataset Europarl

def read_translation(archivo_ingles, archivo_espanol):
    with open(archivo_ingles, 'r', encoding='utf-8') as f_ingles, open(archivo_espanol, 'r', encoding='utf-8') as f_espanol:
        for oracion_ingles, oracion_espanol in zip(f_ingles, f_espanol):
            yield oracion_ingles.strip(), oracion_espanol.strip()


archivo_ingles = 'data/europarl/europarl-v7.es-en.en'
archivo_espanol = 'data/europarl/europarl-v7.es-en.es'

# Leer el conjunto de datos
for i, (ingles, espanol) in enumerate(read_translation(archivo_ingles, archivo_espanol)):
    print('Inglés:', ingles)
    print('Español:', espanol)
    print('---')
    if i == 15:
        break

FileNotFoundError: [Errno 2] No such file or directory: 'data/europarl/europarl-v7.es-en.en'